In [ ]:
%load_ext autoreload
%autoreload 2

import csv
import ipywidgets as widgets
import json
import openai
import os
import pandas as pd
import papermill as pm
import scrapbook as sb
import torch

from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm_notebook
from transformers import AutoModel, AutoTokenizer

tqdm_notebook().pandas()

pd.set_option('display.max_colwidth', None)

In [ ]:
# base_name = "2021_Hashimoto_Neural_ODE_and_holographic_QCD_PUB"
# project_folder = "diygenomics-projects"
# sub_category = "math"
# work_bucket = "AdS-CFT"
# external_id = "2023_05_22_92dc0613b4493d7b5847g"

In [ ]:
data_path = os.getenv('DATA_PATH')
file_path = lambda *args: os.path.join(data_path, project_folder, sub_category, work_bucket,
                                       base_name, 'mathpix', *args)

index_col = 'uuid'
openai_model = 'text-embedding-ada-002'

input_file = 'extracted_annotated_math.csv'
output_file = 'extracted_annotated_math.csv'

In [ ]:
df = pd.read_csv(file_path(input_file), index_col=index_col)

In [ ]:
# df['openai_math_embeddings'] = df['math'].progress_apply(lambda x: openai.Embedding.create(input=x, engine=openai_model, temperature=0,)['data'][0]['embedding'])

In [ ]:
df['openai_sympy_embeddings'] = df['gpt_python_math'].progress_apply(lambda x: openai.Embedding.create(input=x, engine=openai_model, temperature=0,)['data'][0]['embedding'])

In [ ]:
df['openai_clean_math_embeddings'] = df['clean_math'].progress_apply(lambda x: openai.Embedding.create(input=x, engine=openai_model, temperature=0,)['data'][0]['embedding'])

In [ ]:
def get_embedding(text):
    return model.encode(text).tolist()

In [ ]:
model = SentenceTransformer('flax-sentence-embeddings/st-codesearch-distilroberta-base')
# df['codesearch_math_embeddings'] = df['math'].apply(get_embedding)
df['codesearch_sympy_embeddings'] = df['gpt_python_math'].apply(get_embedding)
df['codesearch_clean_math_embeddings'] = df['clean_math'].apply(get_embedding)

In [ ]:
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')
# df['multi_qa_math_embeddings'] = df['math'].apply(get_embedding)
df['multi_qa_sympy_embeddings'] = df['gpt_python_math'].apply(get_embedding)
df['multi_qa_clean_math_embeddings'] = df['clean_math'].apply(get_embedding)

In [ ]:
model_name = 'witiko/mathberta'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
def get_model_embeddings(text):
    if len(text) > 514:
        text = text[:514]
    tokenized_text = tokenizer.tokenize(text)
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    input_ids = torch.tensor([input_ids])
    with torch.no_grad():
        outputs = model(input_ids)
        embeddings = outputs.last_hidden_state
    return embeddings.numpy().tolist()[0][0]

In [ ]:
# df['mathbert_math_embeddings'] = df['math'].apply(get_model_embeddings)
df['mathbert_sympy_embeddings'] = df['gpt_python_math'].apply(get_model_embeddings)
df['mathbert_clean_math_embeddings'] = df['clean_math'].apply(get_model_embeddings)

In [ ]:
df.to_csv(file_path(output_file), quoting=csv.QUOTE_MINIMAL)

In [ ]:
sb.glue('status', 'completed')